# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Preprocessing Data dari data hasil filtering.

**Anggota Kelompok:**  
- 140810230011 - Lukas Austin  
- 140810230045 - Devin Suryadi  
- 140810230057 - Orlando Bloem Sutono  

Mata Kuliah: Data Mining  
Kelas: A  
Dosen: Bu Helen

---

## 1. Import Library
Library yang dibutuhkan untuk preprocessing teks.

In [14]:
import os
import re
import time
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import emoji
import json
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indoNLP.preprocessing import replace_word_elongation, replace_slang
# from indoNLP.preprocessing import remove_stopwords

## 2. Load Dataset
Memuat data hasil filtering komentar dari YouTube.

In [15]:
pd.set_option('display.max_colwidth', 200)

df = pd.read_csv("dataset_filtered.csv")
df = df[['comment']].copy()

print(f"Dataset dimuat: {len(df)} baris, kolom: {list(df.columns)}")
df.head()

Dataset dimuat: 924 baris, kolom: ['comment']


,comment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak."
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu."
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut..."
3,Ayooo pak pur ttp semangat bangun indonesia 👍
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban


## 3. Text Cleaning
Melakukan pembersihan HTML, URL, emoji, tanda baca, dan karakter berulang.

In [16]:
# Menghapus HTML dan tag tidak relevan
def remove_html(text):
    return BeautifulSoup(str(text), "html.parser").get_text()

# Menormalisasi unicode ke bentuk konsisten
def normalize_unicode(text):
    return unicodedata.normalize('NFKC', str(text))

# Menghapus URL, mention, dan hashtag
def remove_url_mentions(text):
    return re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", str(text))

# Menghapus emoji dari teks
def remove_emoji(text):
    return emoji.replace_emoji(str(text), replace=' ')

# Mengubah teks ke lowercase dan menghapus tanda baca
def lowercase_and_remove_punct(text):
    return re.sub(r'[^a-z\s]', ' ', str(text).lower())

# Menghapus kata yang hanya 1 huruf
def remove_single_letter(text):
    text = re.sub(r'\b[a-zA-Z]\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
# Menormalkan spasi berlebih
def normalize_space(text):
    return re.sub(r'\s+', ' ', str(text)).strip()

In [24]:
df['clean_text'] = df['comment'].apply(remove_html)
df['clean_text'] = df['clean_text'].apply(normalize_unicode)
df['clean_text'] = df['clean_text'].apply(remove_url_mentions)
df['clean_text'] = df['clean_text'].apply(remove_emoji)
df['clean_text'] = df['clean_text'].apply(lowercase_and_remove_punct)
df['clean_text'] = df['clean_text'].apply(replace_word_elongation)  #indoNLP
df['clean_text'] = df['clean_text'].apply(remove_single_letter)
df['clean_text'] = df['clean_text'].apply(normalize_space)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja pak purbaya dan bu sri mulyani senang lihat kerja bpk sehat semangat ya pak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat ngak dikerjain sama oligarki para mafia di sektor energy pangan dan keuangan yg merasa kepentingannya terganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyatanya bank himbara masih selektif mengucurkan dana untuk umkm yg sangat membutuhkan tambahan modal seperti yg saya alami punya usaha percetakan sdh berjalan sejak sampai sekarang membutuhkan mo...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak pur ttp semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban


## 4. Normalization
Mengganti kata tidak baku (gaul), typo, dan kata dengan makna yang sama namun berbeda penulisan menjadi kata yang sama dan benar.

##### Daftar kata typo yang ada di dataset melalui eksplorasi dataset secara manual.

In [25]:
with open("typo_dict.json", "r", encoding="utf-8") as f:
    typo_dict = json.load(f)
    
test_data = dict(list(typo_dict.items())[:5])
test_data

{'bpk': 'bapak',
 'ngak': 'tidak',
 'sdh': 'sudah',
 'g': 'enggak',
 'ttp': 'tetap'}

##### Fungsi Normalisasi

In [26]:
def normalize(text):
    text = replace_slang(text)  # ubah kata slang ke bentuk baku dengan indoNLP
    words = text.split()
    normalized = [typo_dict.get(w, w) for w in words]
    return " ".join(normalized)

In [27]:
df['clean_text'] = df['clean_text'].apply(normalize)  # normalisasi typo dan slang
df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja pak purbaya dan bu sri mulyani senang lihat kerja bapak sehat semangat nya pak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat tidak dikerjain saling oligarki para mafia disalurkan sektor energi pangan dan keuangan yang merasa kepentingannya terganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyatanya bank himbara masih selektif mengucurkan dana untuk umkm yang sangat membutuhkan tambahan modal seperti yang saya alami punya usaha percetakan sudah berjalan sejak sampai sekarang membutuh...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas harus mengontrol pemain penggunaan uang kenapa negara seperti rakyatnya miskin pemain banyak rekayasa yang serong ngeri ngeri rakyat jadi korban


## 5. Stopword Removal dan Stemming
Menghapus kata umum yang tidak informatif dan mengembalikan kata ke bentuk dasarnya.

##### Inisialisasi Stopword dan Stemmer

In [28]:
# Inisialisasi Stopword
stop_factory = StopWordRemoverFactory()
stop_remover = stop_factory.create_stop_word_remover()

# Inisialisasi Stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

##### Mendefinisikan fungsi untuk melindungi kata negasi sebelum stemming (karena library Sastrawi biasa menghapusnya), serta fungsi pemulihan setelah stemming selesai dilakukan.

In [29]:
# Fungsi proteksi negasi sebelum stemming karena biasanya library ini menghilangkan kata negasi
def protect_negations(text):
    text = re.sub(r"\btidak\b", "NEG-TIDAK", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbukan\b", "NEG-BUKAN", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbelum\b", "NEG-BELUM", text, flags=re.IGNORECASE)
    text = re.sub(r"\bkurang\b", "NEG-KURANG", text, flags=re.IGNORECASE)
    text = re.sub(r"\btanpa\b", "NEG-TANPA", text, flags=re.IGNORECASE)
    return text

# Fungsi pemulihan negasi setelah stemming
def restore_negations(text):
    text = re.sub(r"\bneg-tidak\b", "tidak", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-bukan\b", "bukan", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-belum\b", "belum", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-kurang\b", "kurang", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-tanpa\b", "tanpa", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

##### Code eksekusi tahap proteksi negasi, penghapusan stopword, stemming, dan pemulihan negasi secara berurutan.

In [30]:
# Proteksi kata negasi
df['clean_text'] = df['clean_text'].apply(protect_negations)

# Stopword removal
df['clean_text'] = df['clean_text'].apply(lambda x: stop_remover.remove(x))

# Stemming
df['clean_text'] = df['clean_text'].apply(lambda x: stemmer.stem(x))

# Pulihkan kata negasi
df['clean_text'] = df['clean_text'].apply(restore_negations)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyata bank himbara selektif kucur dana umkm sangat butuh tambah modal alami punya usaha cetak jalan sejak sekarang butuh modal juta aju kur mikro salur ac padahal pakai jamin malah suruh ambil yan...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas kontrol main guna uang negara rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban


## 6. Menambahkan Kolom Sentimen
Menambahkan kolom `sentiment` kosong sebagai placeholder untuk tahap labeling.

In [31]:
# Tambahkan kolom sentiment kosong
df['sentiment'] = ""

df[['comment', 'clean_text', 'sentiment']].head(5)

,comment,clean_text,sentiment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu,
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyata bank himbara selektif kucur dana umkm sangat butuh tambah modal alami punya usaha cetak jalan sejak sekarang butuh modal juta aju kur mikro salur ac padahal pakai jamin malah suruh ambil yan...,
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas kontrol main guna uang negara rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban,


In [32]:
pd.DataFrame({
    'comment': df['clean_text'],
    'sentiment': df['sentiment']
}).to_csv("dataset_cleaned.csv", index=False, encoding='utf-8')

## 7. Labeling dengan API Gemini
Melakukan labeling otomatis pada seluruh data menggunakan model Gemini 2.0 Flash.
Prompt dibuat agar model memahami konteks kebijakan dan mengklasifikasikan opini publik secara akurat ke dalam tiga kelas sentimen: positif, negatif, dan netral.

##### API client dari Google Gemini.

In [33]:
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)
GEMINI = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI)

##### Fungsi untuk prompt dan menjalankan pelabelan comment

In [38]:
def classify_sentiment_batch(comments):
    # Tambahkan penomoran pada setiap komentar
    numbered_comments = "\n".join([f"{i+1}. {c}" for i, c in enumerate(comments)])

    # Prompt
    prompt = f"""
    Anda adalah analis sentimen publik. 
    Tugas Anda adalah menentukan apakah setiap komentar berikut memiliki sentimen positif, negatif, atau netral 
    terhadap kebijakan Menteri Keuangan Purbaya yang memindahkan dana Rp 200 triliun ke Bank Himbara.
    Tolong baca dengan cermat setiap komentar dan berikan penilaian yang akurat sesuai konteksnya, jangan hanya melihat berdasarkan kata-kata negatif, pahami keseluruhan.
    Jumlah komentar ada {len(comments)}.

    Kriteria penilaian:
    - Positif berarti mendukung, memuji, atau menilai kebijakan ini baik/bermanfaat.
    - Negatif berarti menentang, mengkritik, atau menilai kebijakan ini buruk/merugikan.
    - Netral berarti informatif / tidak informatif, bercanda, tidak menunjukkan opini jelas.

    Berikut daftar komentar yang perlu diklasifikasikan:
    {numbered_comments}

    Kembalikan hasil dalam format Python list seperti berikut:
    ["Positif", "Negatif", "Netral", ...]
    Jumlah elemen dalam list harus sama dengan jumlah komentar ({len(comments)}).
    Jangan tambahkan teks lain, jangan tulis kode, jangan tulis ```python, jangan ada komentar tambahan.
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    raw_output = response.text.strip()

    # Bersihkan tanda ``` jika gemini mengirimkan format yang tidak sesuai
    raw_output = re.sub(r"^```(?:python)?", "", raw_output)
    raw_output = re.sub(r"```$", "", raw_output).strip()

    # Parsing dengan ambil isi dalam tanda kutip
    try:
        labels = re.findall(r'"(.*?)"', raw_output)
        if not labels:  # Jika tidak ada tanda kutip, coba pisah berdasarkan koma
            labels = [w.strip(" []'\"\n") for w in raw_output.split(",") if w.strip()]
        if len(labels) != len(comments):
            print(f"Jumlah label ({len(labels)}) ≠ jumlah komentar ({len(comments)})")
            if len(labels) < len(comments):
                labels += ["Error"] * (len(comments) - len(labels))
            else:
                labels = labels[:len(comments)]
        return labels
    except Exception as e:
        print("Format output tidak valid, hasil mentah:")
        print(raw_output)
        return ["Error"] * len(comments)

##### Persiapan data untuk gemini dengan dataset yang sudah dibersihkan

In [39]:
# Baca dataset_cleaned.csv yang sudah jadi
df_original = pd.read_csv("dataset_cleaned.csv")

# Gunakan kolom 'comment' untuk labeling
print(f"Total komentar untuk labeling: {len(df_original)}")
df_original.head()

Total komentar untuk labeling: 924


,comment,sentiment
0,langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,NaN
1,moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu,NaN
2,nyata bank himbara selektif kucur dana umkm sangat butuh tambah modal alami punya usaha cetak jalan sejak sekarang butuh modal juta aju kur mikro salur ac padahal pakai jamin malah suruh ambil yan...,NaN
3,ayo pak purbaya tetap semangat bangun indonesia,NaN
4,ayo rakyat cerdas kontrol main guna uang negara rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban,NaN


In [40]:
# Test dengan 5 komentar pertama
comments = df_original['comment'].head().tolist()
labels = classify_sentiment_batch(comments)

df_preview = pd.DataFrame({
    "comment": comments,
    "sentiment": labels
})

df_preview

,comment,sentiment
0,langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,Positif
1,moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu,Positif
2,nyata bank himbara selektif kucur dana umkm sangat butuh tambah modal alami punya usaha cetak jalan sejak sekarang butuh modal juta aju kur mikro salur ac padahal pakai jamin malah suruh ambil yan...,Negatif
3,ayo pak purbaya tetap semangat bangun indonesia,Positif
4,ayo rakyat cerdas kontrol main guna uang negara rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban,Negatif


In [41]:
def process_sentiment_labeling(df, column_name, batch_size=25, delay=30):
    all_labels = []  # menyimpan semua label hasil model

    # Loop setiap batch komentar
    for i in range(0, len(df), batch_size):
        # Ambil komentar
        batch_comments = df[column_name].iloc[i:i+batch_size].astype(str).tolist()
        print(f"\nMemproses batch {i//batch_size + 1} ({len(batch_comments)} komentar)...")

        while True:
            try:
                labels = classify_sentiment_batch(batch_comments)
                break  # keluar dari loop jika berhasil
            except Exception as e:
                if "RESOURCE_EXHAUSTED" in str(e) or "429" in str(e):
                    print("Terkena limit API (429). Menunggu 120 detik sebelum mencoba ulang batch ini...")
                    time.sleep(120)
                else:
                    print(f"Terjadi error pada batch {i//batch_size + 1}: {e}")
                    labels = ["Error"] * len(batch_comments)
                    break  # hentikan loop jika error bukan 429

        # Simpan hasil ke list utama
        all_labels.extend(labels)

        # Jeda antar batch agar tidak limit API request
        if i + batch_size < len(df):
            print(f"Menunggu {delay} detik sebelum lanjut ke batch berikutnya.")
            time.sleep(delay)

    df_result = pd.DataFrame({
        "comment": df[column_name],
        "sentiment": all_labels
    })
    
    return df_result

In [42]:
# Jalankan labeling untuk semua data
df_result = process_sentiment_labeling(df_original, 'comment', batch_size=50, delay=120)


Memproses batch 1 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

Memproses batch 2 (50 komentar)...

Memproses batch 2 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

Memproses batch 3 (50 komentar)...

Memproses batch 3 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

Memproses batch 4 (50 komentar)...

Memproses batch 4 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

Memproses batch 5 (50 komentar)...

Memproses batch 5 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

Memproses batch 6 (50 komentar)...

Memproses batch 6 (50 komentar)...
Menunggu 120 detik sebelum lanjut ke batch berikutnya.

In [44]:
# Retry data error
# Ambil baris yang gagal
error_mask = df_result['sentiment'] == "Error"
df_error = df_result[error_mask].copy()

print(f"Jumlah komentar error yang akan diulang: {len(df_error)}")

if len(df_error) > 0:
    # Jalankan ulang labeling hanya untuk data error
    retry_result = process_sentiment_labeling(
        df=df_error,
        column_name="comment",
        batch_size=25, 
        delay=60
    )
    # Gantikan label Error dengan hasil retry
    df_result.loc[error_mask, 'sentiment'] = retry_result['sentiment'].values
    print("Retry selesai. Label Error telah diperbarui.")
else:
    print("Tidak ada data Error. Tidak perlu retry.")

Jumlah komentar error yang akan diulang: 100

Memproses batch 1 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 2 (25 komentar)...

Memproses batch 2 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 3 (25 komentar)...

Memproses batch 3 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 4 (25 komentar)...

Memproses batch 4 (25 komentar)...
Retry selesai. Label Error telah diperbarui.
Retry selesai. Label Error telah diperbarui.


In [46]:
# Simpan hasil labeling ke dataset_labeled.csv
df_result.to_csv("dataset_labeled.csv", index=False, encoding="utf-8")

print(f"Dataset final berhasil disimpan ke dataset_labeled.csv")
print(f"Total baris: {len(df_result)}")
print(f"\nDistribusi sentimen:")
print(df_result['sentiment'].value_counts())

Dataset final berhasil disimpan ke dataset_labeled.csv
Total baris: 924

Distribusi sentimen:
sentiment
Positif    435
Negatif    334
Netral     155
Name: count, dtype: int64
